### Building chatbot with multiple tools using langgraph

### Create a chatbot with tool capabilities with arxiv, wikipedia search and some functions

In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [3]:
from langchain_groq import ChatGroq
llm_groq=ChatGroq(model="llama-3.3-70b-versatile")
llm_groq.invoke("hi I am sachchida. I am AI enthusiast")

/home/sachchida/anaconda3/envs/torch-cuda12_8/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content="Hi Sachchida, nice to meet you. It's great to hear that you're an AI enthusiast. Artificial Intelligence is a fascinating field with endless possibilities and applications. What aspects of AI interest you the most? Are you more into machine learning, natural language processing, computer vision, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 46, 'total_tokens': 108, 'completion_time': 0.158216253, 'prompt_time': 0.002418389, 'queue_time': 0.048710351, 'total_time': 0.160634642}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9ab46265-009b-4b95-89f0-e2292328261e-0', usage_metadata={'input_tokens': 46, 'output_tokens': 62, 'total_tokens': 108})

In [5]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [7]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [8]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [9]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [10]:
wiki.invoke("barack obama")

'Page: Barack Obama\nSummary: Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.\nBorn in Honolulu, Hawaii, Obama graduated from Columbia University in 1983 with a Bachelor of Arts degree in polit'

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults()

/tmp/ipykernel_10472/3543858584.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily=TavilySearchResults()


In [13]:
tavily.invoke("provide the current USD to INR conversion rate")

[{'title': 'US dollar to Indian rupees Exchange Rate History - Wise',
  'url': 'https://wise.com/us/currency-converter/usd-to-inr-rate/history',
  'content': 'The exchange rate for US dollar to Indian rupees is currently 87.8944 today, reflecting a -0.184% change since yesterday. Over the past week, the value of US dollar has remained relatively stable, with a -1.059% decrease compared to its value 7 days ago. [...] During the past week, the exchange rate of US dollar to Indian rupees has fluctuated between a high of 88.8897 on 09-10-2025 and a low of 87.6904 on 16-10-2025. The largest 24-hour price movement occurred on 15-10-2025, with a -0.482% decrease in value.\n\n### USD to INR conversion chart\n\nPercentage change in the last last 1 day\n\n## 0\n\nWe use the real, mid-market rate with no sneaky mark-up to hide the fees.Learn more\n\n## Top currency pairings for USD\n\nPercentage change in the last 24 hours [...] | 1 USD to INR | Last 7 days |\n --- |\n| High | 88.8244 |\n| Low | 

In [14]:
### combine all the tools in the list
tools=[arxiv,wiki,tavily]

In [22]:
### intialize my llm model
from langchain_groq import ChatGroq
llm=ChatGroq(model="openai/gpt-oss-20b")
### bind llm with tools (list of tools)
llm_with_tools=llm.bind_tools(tools)

In [23]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

In [24]:
llm_with_tools.invoke([HumanMessage(content=f"what is the recent AI news")])

AIMessage(content='', additional_kwargs={'reasoning_content': "We need to provide recent AI news. Likely need up-to-date info. Use tavily_search_results_json for current news. Let's search.", 'tool_calls': [{'id': 'fc_c16ba15a-d6ff-42e3-850b-174a3cc34043', 'function': {'arguments': '{"query":"recent AI news 2025 October"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 291, 'total_tokens': 354, 'completion_time': 0.063259625, 'prompt_time': 0.014084271, 'queue_time': 0.043323098, 'total_time': 0.077343896}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d587a02fb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--28b770ac-e89c-4de8-8251-940965784dbc-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent AI news 2025 October'}, 'id': 'fc_c16ba15a-d6ff-42e3-850b-174a3cc34043', 'type': 'tool_call'}], usage_metadata={'

In [25]:
llm_with_tools.invoke([HumanMessage(content=f"what is the recent AI news")]).tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'recent AI news'},
  'id': 'fc_cff7315e-ba3e-45ef-aade-817e1be79a57',
  'type': 'tool_call'}]